In [ ]:
pip install selenium

In [ ]:
#Importing libraries
from selenium import webdriver 
import pandas as pd 
from selenium.webdriver.common.keys import Keys
import time
import numpy as np
from selenium.common.exceptions import NoSuchElementException, NoSuchAttributeException

# creating instance for web driver
driver = webdriver.Chrome(executable_path=r'C:\Program Files (x86)\Google\Chrome\Application\81.0.4044.122\default_apps\chromedriver.exe')

driver.get("https://soundcloud.com/search/sounds?q=lockdown&filter.created_at=last_month&filter.genre_or_tag=drum%20%26%20bass")
time.sleep(10)

def search():
    df = pd.DataFrame(columns = ['trace_title', 'genre','link'])

    elem = driver.find_element_by_tag_name("body")

    # Scrolling the page (no_of_pagedowns = number of scrolls)
    no_of_pagedowns = 500
    while no_of_pagedowns:
        elem.send_keys(Keys.PAGE_DOWN)
        time.sleep(1)
        no_of_pagedowns-=1
 
    # song title and artist name
    trace_data_scrap = driver.find_elements_by_class_name("soundTitle__usernameTitleContainer")

    # song tag
    #song_tag_scrap = driver.find_elements_by_class_name("soundTitle__tag sc-tag sc-tag-small")
    trace_genre_scrap = driver.find_elements_by_xpath("//*[@class='searchTags__tag sc-tag sc-tag-small sc-tag-selected']/span")
       
    dropdown = driver.find_elements_by_css_selector("* > div > div > div > div.sound__content > div.sound__header > div > div > div.soundTitle__usernameTitleContainer > a")
    links = []
    for i in dropdown:
        if len(i.get_attribute("href")) > 0:
            links.append(i.get_attribute("href"))
        else:
            links.append('None')
        
    # creating empty lists for storing scraped data
    trace_data = []
    trace_genre = []
    
    for i in trace_data_scrap:
        if len(i.text) > 0:
            trace_data.append(i.text)
        else:
            trace_data.append('None')

    for i in trace_genre_scrap:
        if len(i.text) > 0:
            trace_genre.append(i.text)
        else:
            trace_genre.append('None')
        
    #exception for range error
    title =[]
    for i in trace_data:
        if i == 'lockdown sessions vol. 1':          
            title.append(i)                          
        else:
            title.append(i.split('\n')[1])      

    print(len(title), len(trace_genre),len(links))

    # creating a temporary dataframe
    index_df = pd.DataFrame.from_dict({"trace_title": title,  
                            "genre":trace_genre*len(title), "link":links},orient = 'index')
    temp_df = index_df.transpose()
    df = df.append(temp_df, ignore_index=True)
    print('dataframe shape is ',df.shape)

    return df
    
data_err = search()

In [ ]:
data_err

In [ ]:
data_err.to_csv(r'C:\Users\margg\Desktop\vitalba\SNA\data_clara_err.csv', index = False)